In [25]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Added CTX to enable better click management.
from dash import ctx


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import mongoDbCrud as mgdbc

import base64

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "test"
shelter = mgdbc.AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_dict(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

imgFileName = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(imgFileName, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='Salvare Logo')),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H1('Zayne Harthun')),
    html.Hr(),
    html.Div(className='buttonRow',
             style={'display':'flex'},
             children=[
                 html.Button(id='submit-button-one', n_clicks=0,children='Water Rescue'),
                 html.Button(id='submit-button-two', n_clicks=0,children='Mountain Rescue'),
                 html.Button(id='submit-button-three', n_clicks=0,children='Disaster Rescue'),
                 html.Button(id='submit-button-four', n_clicks=0,children='Reset')
             ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable='single'
        

    ),
    html.Br(),
    html.Hr(),
    dcc.Graph(id='graph-id'),
    
    html.Div(
        id='map-id',
        className='col s12 m6',
    
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback should manage the custom graph that Mr. Salvare Asked for.
@app.callback(
    Output('graph-id','figure'),
    [Input('submit-button-one','n_clicks'),
     Input('submit-button-two','n_clicks'),
     Input('submit-button-three','n_clicks'),
     Input('submit-button-four','n_clicks')]
)
def update_graph(button1,button2,button3,button4):
    df = pd.DataFrame.from_dict(shelter.read({}))
    fig = px.histogram(df,x='animal_type')
    
    if("submit-button-one" == ctx.triggered_id):
        df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog",
                                                  "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                                                  "sex_upon_outcome": "Intact Female",
                                                  "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}}))
        fig = px.histogram(df,x='breed')
    elif("submit-button-two" == ctx.triggered_id):
        df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog",
                                                  "breed":{"$in":["German Shepherd","Alaskan Malamute", "Old English Sheepdog","Siberian Husky","Rottweiler"]},
                                                  "sex_upon_outcome":"Intact Male",
                                                  "age_upon_outcome_in_weeks":{"$gte":26.0, "$lte":156.0}}))
        fig = px.histogram(df,x='breed')
    elif("submit-button-three" == ctx.triggered_id):
        df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog",
                                                  "breed":{"$in":["Doberman pinscher","German Shepard", "Golden Retriever","Bloodhound","Rottweiler"]},
                                                  "sex_upon_outcome":"Intact Male",
                                                  "age_upon_outcome_in_weeks":{"$gte":20.0, "$lte":300.0}}))
        fig = px.histogram(df,x='breed')
    elif("submit-button-four" == ctx.triggered_id):
        df = pd.DataFrame.from_dict(shelter.read({}))
        fig = px.histogram(df,x='animal_type')
    
    return fig

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if(selected_columns is not None):
        return [{
              'if': { 'column_id': i },
              'background_color': '#D2F3FF'
        } for i in selected_columns]
    else:
        return
    
@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one','n_clicks'),
     Input('submit-button-two','n_clicks'),
     Input('submit-button-three','n_clicks'),
     Input('submit-button-four','n_clicks')])
def on_click(button1,button2,button3,button4):
    #start case
    df = pd.DataFrame.from_dict(shelter.read({}))
    
    if("submit-button-one" == ctx.triggered_id):
        df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog",
                                                  "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                                                  "sex_upon_outcome": "Intact Female",
                                                  "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}}))
    elif("submit-button-two" == ctx.triggered_id):
        df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog",
                                                  "breed":{"$in":["German Shepherd","Alaskan Malamute", "Old English Sheepdog","Siberian Husky","Rottweiler"]},
                                                  "sex_upon_outcome":"Intact Male",
                                                  "age_upon_outcome_in_weeks":{"$gte":26.0, "$lte":156.0}}))
    elif("submit-button-three" == ctx.triggered_id):
        df = pd.DataFrame.from_dict(shelter.read({"animal_type":"Dog",
                                                  "breed":{"$in":["Doberman pinscher","German Shepard", "Golden Retriever","Bloodhound","Rottweiler"]},
                                                  "sex_upon_outcome":"Intact Male",
                                                  "age_upon_outcome_in_weeks":{"$gte":20.0, "$lte":300.0}}))
    elif("submit-button-four" == ctx.triggered_id):
        df = pd.DataFrame.from_dict(shelter.read({}))
        
    #Cleanup and return.
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    #Something something, single row selection.
    if index is None:
        row = 0
    else:
        row = index[0]
        
    return[dl.Map(style={'width': '500px', 'height':'500px'},
                 center = [30.75,-97.48], zoom=10, children =[
                     dl.TileLayer(id="base-layer-id"),
                     dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                              children = [
                                  dl.Tooltip(dff.iloc[row,4]),
                                  dl.Popup([
                                      html.H1("Animal Name"),
                                      html.P(dff.iloc[row,9])
                                  ])
                              ])
                 ])]
   
    

app.run_server(debug=True)

Client defined
Database defined
Collection defined
Dash app running on http://127.0.0.1:29285/
